In [2]:
import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
import flashinfer
from typing import List, Tuple

import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
import flashinfer
from typing import List, Tuple

import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
import flashinfer
from typing import List, Tuple

import os
import json
from typing import List, Tuple
import gc
import torch
import flashinfer
from rich.progress import Progress

from rich import print

In [3]:
device = torch.device(f"cuda:{0}")
workspace_bytes = 128 * 1024 * 1024
workspace = torch.empty(workspace_bytes, dtype=torch.uint8, device=device)
wrapper   = flashinfer.BatchPrefillWithPagedKVCacheWrapper(workspace, "NHD")


In [5]:
rank = 0
print(dict(rank=rank))

batch_size = 4
seq_len = 1024
cp_size = 4
head_dim = 128
print(dict(batch_size=batch_size, seq_len=seq_len, cp_size=cp_size, head_dim=head_dim))

num_layers = 1
num_qo_heads = 16
num_kv_heads = 8
page_size = 16
print(dict(num_qo_heads=num_qo_heads, num_kv_heads=num_kv_heads, page_size=page_size, num_layers=num_layers))

local_seq_len  = seq_len // cp_size
local_seq_len_slice = local_seq_len // 2
nnz_qo_local   = batch_size * local_seq_len
print(dict(local_seq_len=local_seq_len, nnz_qo_local=nnz_qo_local, local_seq_len_slice=local_seq_len_slice))

max_page_index = (seq_len + page_size - 1) // page_size
print(dict(max_page_index=max_page_index))

{'rank': 0}

{'batch_size': 4, 'seq_len': 1024, 'cp_size': 4, 'head_dim': 128}

{'num_qo_heads': 16, 'num_kv_heads': 8, 'page_size': 16, 'num_layers': 1}

{'local_seq_len': 256, 'nnz_qo_local': 1024, 'local_seq_len_slice': 128}

{'max_page_index': 64}

In [6]:
q_at_layer = torch.randn(
    num_layers, nnz_qo_local, num_qo_heads, head_dim,
    dtype=torch.float16, device=device
)

In [23]:
kv_at_layer = torch.randn(
    num_layers, batch_size * max_page_index, 2, page_size, num_kv_heads, head_dim,
    dtype=torch.float16, device=device
)
# [0, seq_len), [seq_len, seq_len * 2), ...

In [24]:
print(dict(q_at_layer=q_at_layer.shape, kv_at_layer=kv_at_layer.shape))

{'q_at_layer': torch.Size([1, 1024, 16, 128]), 'kv_at_layer': torch.Size([1, 256, 2, 16, 8, 128])}

In [ ]:
qo_indptr = torch.arange(
    0, (batch_size * local_seq_len) + 1, local_seq_len // 2,
    dtype=torch.int32, device=device
)
qo_indptr

tensor([   0,  128,  256,  384,  512,  640,  768,  896, 1024], device='cuda:0',
       dtype=torch.int32)

In [25]:
a = [
    i * max_page_index // (2 * cp_size)
    for i in range(1, 2 * batch_size + 1)
]
print(f"{max_page_index=}, page_idx={a}")

# max_page_index // (2 * cp_size)
b = [
    (a[i-1], a[-i])
    for i in range(1, 1+cp_size)
]
print(b)

c = b[rank]
kv_indicies = []
kv_indptrs = [0]
for bs in range(batch_size):
    for this_ctx_len in c:
        m = list(range(0, this_ctx_len))
        kv_indicies.extend(m)
        kv_indptrs.append(kv_indptrs[-1] + len(m))
    pass

print(f"kv_indptrs={kv_indptrs}")
print(f"kv_indicies={kv_indicies}")

max_page_index=64, page_idx=[8, 16, 24, 32, 40, 48, 56, 64]

[(8, 64), (16, 56), (24, 48), (32, 40)]

kv_indptrs=[0, 8, 72, 80, 144, 152, 216, 224, 288]

kv_indicies=[0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 
22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 
12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40,
41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 0, 1, 2, 3, 4, 5, 6, 7,
0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 
31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59,
60, 61, 62, 63, 0, 1, 2, 3, 4, 5, 6, 7, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 
21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49,
50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63]

In [26]:
paged_kv_indptr = torch.tensor(kv_indptrs, dtype=torch.int32, device=device)
paged_kv_indices = torch.tensor(kv_indicies, dtype=torch.int32, device=device)
print(f"paged_kv_indptr={paged_kv_indptr}")
print(f"paged_kv_indices={paged_kv_indices}")

paged_kv_indptr=tensor([  0,   8,  72,  80, 144, 152, 216, 224, 288], device='cuda:0',
       dtype=torch.int32)

paged_kv_indices=tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
        10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
        28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
        46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63,
         0,  1,  2,  3,  4,  5,  6,  7,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
        10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
        28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
        46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63,
         0,  1,  2,  3,  4,  5,  6,  7,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
        10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
        28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
        46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63,
         0,  1,  2,  3,  4,  5,  6,  7,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9,
        10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27,
        28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45,
        46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63],
       device='cuda:0', dtype=torch.int32)

In [27]:
paged_kv_last_len = torch.tensor([page_size] * batch_size * 2, dtype=torch.int32, device=device)
print(f"paged_kv_last_len={paged_kv_last_len}")

paged_kv_last_len=tensor([16, 16, 16, 16, 16, 16, 16, 16], device='cuda:0', dtype=torch.int32)

In [ ]:
wrapper.plan(
    qo_indptr,
    paged_kv_indptr,
    paged_kv_indices,
    paged_kv_last_len,   
    num_qo_heads,
    num_kv_heads,
    head_dim,
    page_size,
    causal=True,
)
